# NX31 - Data Project

### Introduction

In the following data project, we  will be analyzing the public sector's distribution of workforce and compare it to the privat sector's. <br />
We will look into if the distribution of work force has changed since the financial crisis in 2008.

During the analysis, we will be doing the following.
* Etablish an API connection to feed the analysis data, from an external source.
* Clean and structure this data.
* Plot the data.
* Make an analysis and compare with the private sector.


### Packages imported

We will load in the following packages for this data project.

In [1]:
%load_ext autoreload
%autoreload 2

#these two lines above makes sure that the modules are automatically reloaded each time we run a cell

In [2]:
import numpy as np # Imports the numpy package
import matplotlib.pyplot as plt # Imports the matplotlib package from pyplot 
import pandas as pd # Imports the pandas dataframe package
import pydst # Imports the PyDST package

### Data

We have choose to use the PyDST package that can load data from Statistics Denmark using an API call. We put in the table id, that we want data from, and the package then returns the data from Statistics Denmark. </br>
We start off by looking into, what variables that are present in the table. We use this information when we load in the data, to only get the data we want. 

In [3]:
dst = pydst.Dst(lang='en') # creates a variable that sets the pydst package to load data variables in english.

FTEG_variables = dst.get_variables(table_id = 'OBESK2') #creates a variable that contains the variables from the table we look at
FTEG_variables # shows the variable

,elimination,id,text,time,values
0,True,SEKTOR,sector,False,"[{'id': '1032', 'text': 'General government'},..."
1,False,Tid,time,True,"[{'id': '2008K1', 'text': '2008Q1'}, {'id': '2..."


In [4]:
FTEG_variables['values'][0][:5] # shows the sectors in the table

[{'id': '1032', 'text': 'General government'},
 {'id': '1015', 'text': 'Central government'},
 {'id': '1020', 'text': 'Regional government'},
 {'id': '1025', 'text': 'Municipal government'},
 {'id': '1030', 'text': 'Social security funds'}]

### Creating Dictionaires

We create the following dictionaries to better structure the data and to create shorter or more precise variable names. </br>
We again use the variable names from above to make this dictionaires.

In [5]:
columns_dict = {} # creates a empty dictionairy, we fills the dictionary with keys and corosponding values.
# We use the variables seen earlier.
columns_dict['SEKTOR'] = 'Sector' 
columns_dict['TID'] = 'Quarter'
columns_dict['INDHOLD'] = 'FTE'

var_dict = {} # creates a empty dictionairy, we fills the dictionary with keys and corosponding values.
# We use the values seen earlier.
var_dict['Central government'] = 'Central'
var_dict['Regional government'] = 'Regional'
var_dict['Municipal government'] = 'Municipal'
var_dict['Social security funds'] = 'Social security funds'
var_dict['General government'] = 'Total'

### Importing Data

We now import data from Statistic Denmark using the PyDST. We use the variables we found above, so we only get the data that we need.
In this case we need all data from the table.

In [ ]:
FTEG_raw = dst.get_data(table_id = 'OBESK2', variables = {'SEKTOR':['*'], 'Tid':['*']})
# Creates a variable that contains data from the table we put in.
FTEG_raw.head(20) #shows the first 20 data points in the table.

### Renaming variables
We use the dictionaries to rename the variables names.

In [ ]:
FTEG_raw.rename(columns = columns_dict, inplace = True) # renames the columns, using the keys,
# which are the current columns names, and use the values in the dictionary instead

for key, value in var_dict.items():
    FTEG_raw.Sector.replace(key, value, inplace = True) # runs a for loop to replace the variable names,
    # matching the keys and values from the var_dict

FTEG_raw.head(20)

### Table

We create this table to give a better view of the data, with the quaters as index.

In [ ]:
FTEG = FTEG_raw.copy().pivot(index = 'Quarter', columns = 'Sector', values = 'FTE')
# creates a copy and pivot the data using the 'Quarter' as the index.
FTEG.head(20)

### Descirbetive statistics



In [ ]:
FTEG_raw.groupby(['Sector']).describe() # Gives a table of decribtive statistics for analysis.
# Showing the mean, the std., the minimum, the quartiles and the maximum

### Plot of public sectors

In [ ]:
fig1, ax = plt.subplots(figsize=(14,14)) #creates a figure with only one subplot
FTEG_raw['Quarter'] = pd.to_datetime(FTEG_raw['Quarter']) # changes variable Quaters to a time format
# that pandas recognizes
FTEG_raw.set_index('Quarter').groupby('Sector')['FTE'].plot(x="Quarter", legend=True, ax=ax);
# sets index to new Quaters and plots the sectors values on y-axis
ax.set_ylabel("Full-time employed") # sets title for y-axis
ax.set_xlabel('Quaters') # sets title for x-axis
ax.set_title("Public full-time employed") # sets title for the figure

Looking at the plot, it seems the development for total public employment is primarily is driven by changes in the municipalities. To confirm we will have to look at he relative changes.

### Relative changes and merging

Now we will look at the relative development of each sector.
We create four dataframes, one for each sector, where we devide the specific sector with the total public employment.

In [ ]:
rc_central = pd.DataFrame(data=FTEG['Central']/FTEG['Total']*100)
rc_municipal = pd.DataFrame(FTEG['Municipal']/FTEG['Total']*100)
rc_regional = pd.DataFrame(FTEG['Regional']/FTEG['Total']*100)
rc_social = pd.DataFrame(FTEG['Social security funds']/FTEG['Total']*100)
# the four lines of code above creates a pandas dataframe for the relative development,
# and we multiply the 100 to get our result in percentage.

In order to plot the relative development later, we need to merge the four dataframes into one. We do this in the following code.

In [ ]:
rFTEG_1 = pd.merge(rc_central, rc_municipal, on='Quarter') # merging the dataframe for relative central and
# municipalty development changes, and thereby creating a new dataframe contianing relative central and
# municipalty development changes
rFTEG_2 = pd.merge(rFTEG_1, rc_regional, on='Quarter') # we merge the new dataframe with the datafram for
# relative regionale development changes
rFTEG = pd.merge(rFTEG_2, rc_social, on='Quarter') # and again with social

rFTEG.head(20) # we plot our new dataframe, but observe that the variable names are not correct,
# we will solve this in the following code

### Renaming


We rename the colums for our new dataframe with relative development.

In [ ]:
rFTEG.columns = ['Central', 'Municipal', 'Regional', 'Social']

rFTEG.head(20)

The following code needs to be run twice in order to get the quarters to show correctly, we are not sure why.

In [ ]:
fig2, ax = plt.subplots(figsize=(14,14))
rFTEG.plot(legend=True, ax=ax);

rFTEG.reset_index(drop=True)
FTEG_raw.set_index('Quarter')
rFTEG.index = pd.DatetimeIndex(data=rFTEG.index)

ax.set_xlabel("Quarters")
ax.set_ylabel("Percent")
ax.set_title("Share of total public employment")

# move the legend, put numbers on the x-axis, maybe ""%"" on y-axis


### Interactive Figure for relative development - "return !!"

In [ ]:
import ipywidgets as widgets # imports the widget for interactive figures

list(rFTEG.columns.values)

In [ ]:
# rFTEG = rFTEG.reset_index()
# list(rFTEG.columns.values)

In [ ]:
# def plot_e(dataframe, Central):
#     I = rFTEG['Central'] == Central
#     
#     ax = dataframe.loc[I,:].plot()
# 
# widgets.interact(plot_e, 
#                  dataframe = widgets.fixed(rFTEG),
#                  Central = widgets.Dropdown(description='Sector', option=rFTEG.columns.unique())
#                 );

### New data

To compare the public employment to that of the private sector, we need data for the private sector. We have found a table that has this, and will now do the same precedings as we did with the data for the public sector.

In [ ]:
pr_variables = dst.get_variables(table_id = 'LBESK21')
pr_variables

### Dictionaries

In [ ]:
columns_dict3 = {}
columns_dict3['TAL'] = 'remove one of them? - lønmodtagere'
columns_dict3['SEKTOR'] = 'Sector'
columns_dict3['TID'] = 'Quarter'
columns_dict3['INDHOLD'] = 'FTE'

var_dict2 = {}
var_dict2['All sectors'] = 'Total'
var_dict2['Central government'] = 'Central'
var_dict2['Regional government'] = 'Regional'
var_dict2['Municipal government'] = 'Municipal'
var_dict2['Social security funds'] = 'Social'
var_dict2['Public corporations'] = 'Public corp'
var_dict2['Private corporations'] = 'Private corp'
var_dict2['Private nonprofit organizations'] = 'Private NPO'
var_dict2['Sector not stated'] = 'SNS'

"get data"

In [ ]:
EM_raw = dst.get_data(table_id = 'LBESK21', variables = {'TAL':['*'], 'SEKTOR':['*'], 'Tid':['*']})
EM_raw

"rename"

In [ ]:
EM_raw.rename(columns = columns_dict3, inplace = True)

for key, value in var_dict2.items():
    EM_raw.Sector.replace(key, value, inplace = True)

EM_raw

"describe"

In [ ]:
EM_raw.groupby(['Sector']).describe()

"compare public and private to total employment"

"" skriv altid slut kommentar så vi ved hvor vi er og om det er den rigtige""
